# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import keras

In [0]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [74]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [0]:
import numpy as np
trainX1 = x_train[np.extract((y_train<5), x_train)]
trainY1 = y_train[y_train<5]

trainX2 = x_train[np.extract((y_train>=5), x_train)]
trainY2 = y_train[y_train>=5]

# Test data
testX1 = x_test[np.extract((y_test<5), x_test)]
testY1 = y_test[y_test<5]

testX2 = x_test[np.extract((y_test>=5), x_test)]
testY2 = y_test[y_test>=5]

In [76]:
print(trainX1.shape)
print(trainY1.shape)
print(trainX2.shape)
print(trainY2.shape)
print(testX1.shape)

(25000, 32, 32, 3)
(25000,)
(25000, 32, 32, 3)
(25000,)
(5000, 32, 32, 3)


In [0]:
trainX1 = np.array(trainX1).astype('float32')
testX1 = np.array(testX1).astype('float32')

trainX1 /= 255
testX1 /= 255

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
trainY1 = tf.keras.utils.to_categorical(trainY1, num_classes=5)
testY1 = tf.keras.utils.to_categorical(testY1, num_classes=5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

In [0]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,vertical_flip=True,zoom_range=0.3)

In [0]:
#Clear any existing model in memory
tf.keras.backend.clear_session()

#Initialize model, reshape & normalize data
model = tf.keras.models.Sequential()

#Add first convolutional layer    
model.add(tf.keras.layers.Conv2D(32,kernel_size= (3, 3), activation = 'relu',input_shape=(32,32,3)))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add Second convolutional layer    
model.add(tf.keras.layers.Conv2D(32,kernel_size=(3, 3), activation = 'relu'))

#Add maxpool layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Add a dropout layer
model.add(tf.keras.layers.Dropout(0.2))

In [0]:
#Add third convolutional layer
model.add(tf.keras.layers.Conv2D(64,kernel_size= (3, 3),activation='relu'))

#Add maxpool layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add fourth convolutional layer
model.add(tf.keras.layers.Conv2D(64,kernel_size= (3,3),activation='relu'))

#Add max pool layer
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

#Add a dropout layer
#model.add(tf.keras.layers.Dropout(0.2))

In [0]:
#Flatten out the data
model.add(tf.keras.layers.Flatten())

# Adding a dense layer
model.add(tf.keras.layers.Dense(128,activation = 'relu'))

#Adding a drouput layer
model.add(tf.keras.layers.Dropout(0.2))

#Adding output layer
model.add(tf.keras.layers.Dense(5,activation='softmax'))

In [0]:
sgd = tf.keras.optimizers.SGD(lr = 0.1, decay=1e-6, momentum=0.7,nesterov=True)

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [86]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0

In [0]:
#Define some parameters
img_size = 60
img_depth = 3

In [0]:
#Build training generator. 
train_generator = datagen.fit(trainX1)

#Build test generator
test_generator = datagen.fit(testX1)

In [89]:
model.fit_generator(datagen.flow(trainX1, trainY1, batch_size=32),
                    steps_per_epoch=len(trainX1) / 32, epochs=10)

  ...
    to  
  ['...']
Train for 781.25 steps
Epoch 1/10
782/781 [==============================] - 87s 112ms/step - loss: 1.6321 - accuracy: 0.2007
Epoch 2/10
782/781 [==============================] - 86s 110ms/step - loss: 1.6104 - accuracy: 0.1973
Epoch 3/10
782/781 [==============================] - 85s 109ms/step - loss: 1.6098 - accuracy: 0.1969
Epoch 4/10
782/781 [==============================] - 86s 109ms/step - loss: 1.6096 - accuracy: 0.1995
Epoch 5/10
782/781 [==============================] - 86s 109ms/step - loss: 1.6096 - accuracy: 0.1974
Epoch 6/10
782/781 [==============================] - 85s 109ms/step - loss: 1.6102 - accuracy: 0.1948
Epoch 7/10
782/781 [==============================] - 86s 110ms/step - loss: 1.6103 - accuracy: 0.2011
Epoch 8/10
782/781 [==============================] - 85s 109ms/step - loss: 1.6098 - accuracy: 0.2006
Epoch 9/10
782/781 [==============================] - 85s 109ms/step - loss: 1.6096 - accuracy: 0.1984
Epoch 10/10
782/781 [====

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [90]:
model.trainable = False
for layer in model.layers:
    model.trainable = False
    
import pandas as pd
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbfac1dd908>,conv2d,False
1,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fbfac1e6828>,batch_normalization,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbfac1e6eb8>,conv2d_1,False
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fbfac1dae80>,max_pooling2d,False
4,<tensorflow.python.keras.layers.core.Dropout object at 0x7fbfaa1915c0>,dropout,False
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbfac1e6080>,conv2d_2,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fbfac1e6048>,max_pooling2d_1,False
7,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fbfaa191668>,batch_normalization_1,False
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbfaa12ef60>,conv2d_3,False
9,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fbfaa1314e0>,max_pooling2d_2,False


In [91]:
set_trainable = False
for layer in model.layers:
    if layer.name in ['dense_1', 'dense']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbfac1dd908>,conv2d,False
1,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fbfac1e6828>,batch_normalization,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbfac1e6eb8>,conv2d_1,False
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fbfac1dae80>,max_pooling2d,False
4,<tensorflow.python.keras.layers.core.Dropout object at 0x7fbfaa1915c0>,dropout,False
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbfac1e6080>,conv2d_2,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fbfac1e6048>,max_pooling2d_1,False
7,<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fbfaa191668>,batch_normalization_1,False
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fbfaa12ef60>,conv2d_3,False
9,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fbfaa1314e0>,max_pooling2d_2,False


In [92]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [95]:
model.output

<tf.Tensor 'dense_1/Identity:0' shape=(None, 5) dtype=float32>

In [0]:
#get Output layer of Pre0trained model
x = model.output

x1 = tf.keras.layers.Reshape((32,32,3),input_shape=(32,32))(x)
#x1= tf.keras.backend.expand_dims(trainX1,axis=-1)
#Flatten the output to feed to Dense layer
x2 = tf.keras.layers.GlobalAveragePooling2D()(x1)

#Add output layer
prediction = tf.keras.layers.Dense(5,activation='softmax')(x2)

In [0]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [0]:
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [106]:
final_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496 

In [0]:
#Build training generator. 
train_generator = datagen.fit(trainX2)

#Build test generator
test_generator = datagen.fit(testX2)

In [112]:
final_model.fit_generator(datagen.flow(trainX2, trainY2, batch_size=32),
                    steps_per_epoch=len(trainX2) / 32, epochs=10,)

  ...
    to  
  ['...']
Train for 781.25 steps
Epoch 1/10
  1/781 [..............................] - ETA: 1:02

InvalidArgumentError: ignored

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [0]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [0]:
twenty_train.data[0:5]

['Does anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.',
 "Hi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the problem:\n\tI have a rectangular mesh in the uv domain, i.e  the mesh is a \n\tmapping of a 3d Bezier patch into 2d. The area in this domain\n\twhich is inside a trimming loop had to be rendered. The trimming\n\tloop is a set of 2d Bezier curve segments.\n\tFor the sake of notation: the mesh is made up of cells.\n\n\tMy problem is this :\n\tThe trimming area has to be split up into individual smaller\n\tcells bounded by the trimming curve segments. If a cell\n\tis wholly inside the area...then it is output as a whole ,\n\telse it is trivially rejected. \n\n\tDoes any body know how thiss can be done,

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd

In [0]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=0.001, max_df=0.20)
vectors = vectorizer.fit_transform(twenty_train.data)
vectors_test = vectorizer.transform(twenty_test.data)
x1 = vectors
y1 = twenty_train.target
x2 = vectors_test
y2 = twenty_test.target
x = np.array(np.r_[x1.todense(), x2.todense()])
y = np.r_[y1, y2]
#return x, y

In [0]:
pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())

,00,000,01,02,03,0358,04,05,06,07,08,09,10,100,1000,101,101010,1024x768,103,104,105,109,10th,11,110,111,1111,114,118,12,120,1200,121,128,129,13,130,1300,131,134,...,xerox,xian,xians,xlib,xloadimage,xt,xtian,xtianity,xv,xview,yale,ye,yea,yeah,year,years,yeast,yellow,yep,yes,yesterday,yogurt,york,young,younger,youngsters,yourselfers,youth,yr,yrs,yugoslavia,yuv,zaphod,zealand,zero,zillions,zinc,zip,zoom,zooming
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.048614,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2253,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.044298,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.105289,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.053691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2254,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.044341,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2255,0.0,0.0,0.0,0.0,0.0,0.0,0.027729,0.0,0.0,0.0,0.0,0.0,0.009598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0104,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.013864,0.0,0.010664,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [0]:
# use logistic regression with text column only
logreg = LogisticRegression(C=1e9)
logreg.fit(vectors, y1)
y_pred_class = logreg.predict(vectors_test)
print(metrics.accuracy_score(y2, y_pred_class))

0.8189081225033289
